In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

2025-12-04 21:37:08.878637: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-04 21:37:08.909071: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-04 21:37:09.695359: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
ven_info = pd.read_csv("image_data/venomous_status_metadata.csv",
                       names=["nid", "class_id", "ven"], header=0)
train_info = pd.read_csv("image_data/train_images_metadata.csv", index_col=0)

relevant = train_info[["image_path", "class_id"]].merge(ven_info, on="class_id")
n_classes = ven_info['class_id'].nunique()
print(f"Number of classes: {n_classes}")

Number of classes: 296


In [8]:
# from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

BATCH_SIZE = 32
IMG_SIZE = 480
def load_and_preprocess1(img_path, y1, img_size=(IMG_SIZE, IMG_SIZE), onehot=True):
    img = tf.io.read_file("image_data/train_images_large/" + img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = preprocess_input(img)
    if onehot: y1 = tf.one_hot(y1, depth=n_classes)
    return img, y1

def make_dataset1(df, what):
    slices = (df['image_path'].values, df[what].values)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    lp = load_and_preprocess1
    if what=="ven": lp = lambda x, y: load_and_preprocess1(x, y, onehot=False)
    ds = ds.map(lp, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds


In [9]:
train_paths, test_val_paths = train_test_split(relevant, test_size=0.3, random_state=42) # does shuffle
val_paths, test_paths = train_test_split(test_val_paths, test_size=0.33, random_state=42) # does shuffle
cid_train_ds = make_dataset1(train_paths, "nid")
cid_train_ds.cache()
cid_val_ds = make_dataset1(val_paths, "nid")
cid_val_ds.cache()
cid_test_ds = make_dataset1(test_paths, "nid")
ven_train_ds = make_dataset1(train_paths, "ven")
ven_val_ds = make_dataset1(val_paths, "ven")
ven_test_ds = make_dataset1(test_paths, "ven")

In [5]:
from tensorflow.keras.layers import Flatten, Conv2D, Dense, Input, MaxPooling2D, Dropout, Concatenate
from tensorflow.keras import losses
from tensorflow.keras import Model

In [6]:
class WeightedBinaryCrossentropy(losses.BinaryCrossentropy):
    def call(self, y_true, y_pred):
        l = super().call(y_true, y_pred)
        weights = y_true * 0.8 + (1 - y_true) * 0.2
        return tf.reduce_mean(l * weights)

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, Model, losses, callbacks
from tensorflow.keras.applications import EfficientNetV2M

# ======= Data augmentation =======
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
])

dynamic_augment = tf.keras.Sequential([
    layers.RandomRotation(factor=0.1),
    layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomContrast(factor=0.2),
])

# ======= Base model =======
base_model = EfficientNetV2M(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Phase 1: freeze everything

# ======= Build custom head =======
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(1024, activation="relu")(x)
outputs = layers.Dense(n_classes, activation="softmax", name='class_id')(x)

model = Model(inputs=inputs, outputs=outputs)

# ======= Compile Phase 1 =======
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

skip = True
if skip:
    model.load_weights('best_model_phase1.keras')
else:
    # ======= Callbacks =======
    early_stop = callbacks.EarlyStopping(monitor='val_accuracy', patience=0, restore_best_weights=True)
    checkpoint = callbacks.ModelCheckpoint('best_model_phase1.keras', monitor='val_accuracy', save_best_only=True)
    
    # ======= Phase 1: Train top layers =======
    history_phase1 = model.fit(cid_train_ds, validation_data=cid_val_ds, epochs=7, callbacks=[early_stop, checkpoint])

# ======= Phase 2: Layer-wise fine-tuning =======
N_TOP_LAYERS_TO_UNFREEZE = 64  # Example: Unfreeze the last x layers
base_model.trainable = True  # Must be True to allow individual layers to be unfrozen
total_layers = len(base_model.layers)
print(total_layers, N_TOP_LAYERS_TO_UNFREEZE)

for i, layer in enumerate(base_model.layers):
    if i >= total_layers - N_TOP_LAYERS_TO_UNFREEZE:
        # Unfreeze the last N layers
        layer.trainable = True
    else:
        # Freeze the layers before the last N
        layer.trainable = False
# Opció: alacsonyabb learning rate a finomhangoláshoz
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Új checkpoint és early stopping Phase 2-hoz
checkpoint_phase2 = callbacks.ModelCheckpoint('best_model_phase2.keras', monitor='val_accuracy', save_best_only=True)
early_stop_phase2 = callbacks.EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)

# ======= Phase 2: Train the entire model =======
history_phase2 = model.fit(
    cid_train_ds,
    validation_data=cid_val_ds,
    epochs=10,  # finomhangolásnál hosszabb epoch-szám is lehet
    callbacks=[checkpoint_phase2, early_stop_phase2]
)

740 64
Epoch 1/10
1165/1454 ━━━━━━━━━━━━━━━━━━━━ 1:51 385ms/step - accuracy: 0.5161 - loss: 1.8878

2025-12-04 21:58:53.237570: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


1453/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step - accuracy: 0.5258 - loss: 1.8343

2025-12-04 22:00:53.044554: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_25226', 12 bytes spill stores, 12 bytes spill loads

2025-12-04 22:00:53.074406: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36954', 4 bytes spill stores, 4 bytes spill loads

2025-12-04 22:00:59.520636: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-04 22:00:59.651259: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-04 22:01:00.401012: E external/local_xla/x

1454/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step - accuracy: 0.5258 - loss: 1.8342

...

2025-12-04 22:35:21.691153: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


1454/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step - accuracy: 0.8104 - loss: 0.6308

Corrupt JPEG data: 558 extraneous bytes before marker 0xd9


1454/1454 ━━━━━━━━━━━━━━━━━━━━ 732s 503ms/step - accuracy: 0.8203 - loss: 0.6035 - val_accuracy: 0.5967 - val_loss: 1.8863
Epoch 5/10
 627/1454 ━━━━━━━━━━━━━━━━━━━━ 5:26 394ms/step - accuracy: 0.8590 - loss: 0.4753

KeyboardInterrupt: 

In [13]:
N_TOP_LAYERS_TO_UNFREEZE = 128  # Example: Unfreeze the last x layers
base_model.trainable = True  # Must be True to allow individual layers to be unfrozen
total_layers = len(base_model.layers)
print(total_layers, N_TOP_LAYERS_TO_UNFREEZE)

for i, layer in enumerate(base_model.layers):
    if i >= total_layers - N_TOP_LAYERS_TO_UNFREEZE:
        # Unfreeze the last N layers
        layer.trainable = True
    else:
        # Freeze the layers before the last N
        layer.trainable = False
# Opció: alacsonyabb learning rate a finomhangoláshoz
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Új checkpoint és early stopping Phase 2-hoz
checkpoint_phase2 = callbacks.ModelCheckpoint('best_model_phase3.keras', monitor='val_accuracy', save_best_only=True)
early_stop_phase2 = callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

# ======= Phase 2: Train the entire model =======
history_phase2 = model.fit(
    cid_train_ds,
    validation_data=cid_val_ds,
    epochs=10,  # finomhangolásnál hosszabb epoch-szám is lehet
    callbacks=[checkpoint_phase2, early_stop_phase2]
)

740 128
Epoch 1/10
1165/1454 ━━━━━━━━━━━━━━━━━━━━ 2:05 436ms/step - accuracy: 0.8424 - loss: 0.5378

2025-12-04 22:53:37.470924: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


1453/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.8403 - loss: 0.5427

2025-12-04 22:55:51.610564: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-04 22:55:51.752882: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


1454/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - accuracy: 0.8403 - loss: 0.5427

...

1454/1454 ━━━━━━━━━━━━━━━━━━━━ 845s 552ms/step - accuracy: 0.8315 - loss: 0.5636 - val_accuracy: 0.6053 - val_loss: 1.9007
Epoch 2/10
1165/1454 ━━━━━━━━━━━━━━━━━━━━ 2:07 441ms/step - accuracy: 0.9049 - loss: 0.3277

2025-12-04 23:07:06.432517: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: extra compressed data


1454/1454 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.9029 - loss: 0.3324

Corrupt JPEG data: 558 extraneous bytes before marker 0xd9


1454/1454 ━━━━━━━━━━━━━━━━━━━━ 773s 532ms/step - accuracy: 0.8941 - loss: 0.3524 - val_accuracy: 0.6090 - val_loss: 1.9516


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, losses
from tensorflow.keras.applications import EfficientNetV2M

# ======= 0. Data augmentation =======
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    # layers.RandomRotation(factor=0.1),
])

dynamic_augment = tf.keras.Sequential([
    layers.RandomRotation(factor=0.1),
    layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomContrast(factor=0.2),
])

# ======= 1. Load the model without the classification head =======
base_model = EfficientNetV2M(
    weights='imagenet',
    include_top=False,  # This is crucial for feature extraction
    input_shape=(IMG_SIZE, IMG_SIZE, 3)  # Specify input size
)

# ======= 2. Freeze / unfreeze top layers =======
N_TOP_LAYERS_TO_UNFREEZE = 64  # Example: Unfreeze the last x layers
base_model.trainable = True  # Must be True to allow individual layers to be unfrozen
total_layers = len(base_model.layers)
print(total_layers, N_TOP_LAYERS_TO_UNFREEZE)

for i, layer in enumerate(base_model.layers):
    if i >= total_layers - N_TOP_LAYERS_TO_UNFREEZE:
        # Unfreeze the last N layers
        layer.trainable = True
    else:
        # Freeze the layers before the last N
        layer.trainable = False

# ======= 3. Build custom classification head =======
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True)  # Pass the inputs through the partially frozen base
x = layers.GlobalAveragePooling2D()(x)  # Apply pooling
x = layers.Dropout(0.3)(x)  # Add a small dropout for regularization
x = layers.Dense(1024, activation="relu")(x)
class_output = layers.Dense(n_classes, activation="softmax", name='class_id')(x)

cid_model = Model(inputs=inputs, outputs=class_output, name='cid_model')

# ======= 4. Compile the model =======
cid_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = cid_model.fit(cid_train_ds, validation_data=cid_val_ds, epochs=10, verbose=True)

Epoch 1/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 836s 1s/step - accuracy: 0.1658 - loss: 4.0908 - val_accuracy: 0.3270 - val_loss: 2.8197
Epoch 2/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 621s 855ms/step - accuracy: 0.3511 - loss: 2.6821 - val_accuracy: 0.3841 - val_loss: 2.5782
Epoch 3/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 624s 858ms/step - accuracy: 0.4564 - loss: 2.1395 - val_accuracy: 0.4087 - val_loss: 2.5136
Epoch 4/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 624s 858ms/step - accuracy: 0.5523 - loss: 1.6687 - val_accuracy: 0.4226 - val_loss: 2.6135
Epoch 5/10
171/727 ━━━━━━━━━━━━━━━━━━━━ 6:30 702ms/step - accuracy: 0.6285 - loss: 1.3597

KeyboardInterrupt: 